In [2]:
import numpy as np
def RFF(X, D, sameFunction):
    np.random.seed(0)
    n_features = len(X)
    
    if sameFunction:
        omegas1 = (np.random.normal(size=(1, D)))
        omegas = omegas1
        
        for i in range(n_features-1):
            omegas = np.vstack([omegas, omegas1])
            
    else:
        omegas = (np.random.normal(size=(n_features, D)))
        
    b = np.random.uniform(0, 2 * np.pi, size=(D,1))
    X_new = np.transpose(omegas)@X+b
    X_new = np.cos(X_new)
    X_new *= np.sqrt(2) / np.sqrt(D)
    
    return X_new

In [33]:
import pandas as pd

df = pd.read_csv("../data/non_linear_ts_lags.csv")
df_short = df.iloc[:14]

In [34]:
df_short

,lag_7,lag_6,lag_5,lag_4,lag_3,lag_2,lag_1
0,1.000000,1.286629,0.555313,0.325428,-0.313727,0.888525,-0.418892
1,1.286629,0.555313,0.325428,-0.313727,0.888525,-0.418892,1.858173
2,0.555313,0.325428,-0.313727,0.888525,-0.418892,1.858173,1.547551
3,0.325428,-0.313727,0.888525,-0.418892,1.858173,1.547551,-1.087081
4,-0.313727,0.888525,-0.418892,1.858173,1.547551,-1.087081,0.875371
5,0.888525,-0.418892,1.858173,1.547551,-1.087081,0.875371,-0.309091
6,-0.418892,1.858173,1.547551,-1.087081,0.875371,-0.309091,0.053564
7,1.858173,1.547551,-1.087081,0.875371,-0.309091,0.053564,1.609044
8,1.547551,-1.087081,0.875371,-0.309091,0.053564,1.609044,-2.212750
9,-1.087081,0.875371,-0.309091,0.053564,1.609044,-2.212750,-2.656689


In [29]:
A = RFF(df_short.T, 2, False).T.to_numpy()

In [30]:
Q, R = np.linalg.qr(A, mode="complete")

In [31]:
R

array([[-2.71285332, -0.08280552],
       [ 0.        ,  2.78271012],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])

In [19]:
A_0 = A[:1,:]
pinv_A_0 = np.linalg.pinv(A[:1,:])

In [20]:
R_inv = np.linalg.pinv(R)
R_inv

array([[ 3.86595400e-01, -5.70421110e-02],
       [ 2.77555756e-17,  3.81432168e-01]])

In [140]:
a = R_inv.T @ A_0.T

In [141]:
s = 1 + a.T @ a
new_R_inv = R_inv - (1/s) * R_inv @ a @ a.T

In [16]:
Q, R = np.linalg.qr(A[1:,:], mode="reduced")
ts = np.linalg.pinv(np.linalg.pinv(R))
ts

array([[-2.58610175, -0.37942177],
       [ 0.        ,  2.59886785]])

In [14]:
ts


array([[-2.58610175, -0.37942177],
       [ 0.        ,  2.59886785]])

In [143]:
new_R_inv

array([[ 0.38658102, -0.05713136],
       [-0.0010465 ,  0.3749364 ]])

In [134]:
new_R_inv

array([[ 0.38658102, -0.05713136,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.0010465 ,  0.3749364 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [111]:
import scipy

P = scipy.linalg.pinv(A)
#proj_M = P @ A

In [62]:
A[:1,:].shape

(1, 200)

In [69]:
A[:1,:].shape

(1, 200)

In [67]:
tempi = pinv_A_0 @ A[:1,:]
tempi.shape

(200, 200)

In [64]:
(P @ A) @ pinv_A_0

array([[ 1.03824809e-01],
       [-1.02605530e-01],
       [ 5.41644990e-02],
       [ 1.00372494e-01],
       [ 5.05606869e-02],
       [ 2.64504153e-02],
       [ 8.58378696e-02],
       [-1.03737839e-01],
       [ 3.43021183e-02],
       [ 5.89081731e-02],
       [-1.01926070e-01],
       [-4.15050210e-03],
       [-8.98820884e-03],
       [ 2.96748413e-03],
       [ 7.07864403e-02],
       [-3.62700790e-02],
       [-8.65409095e-02],
       [ 1.01511644e-01],
       [ 6.08858799e-02],
       [-9.14101919e-02],
       [-9.36788493e-02],
       [-7.23195486e-02],
       [ 7.96486533e-02],
       [-1.04203268e-01],
       [ 1.53796743e-03],
       [-4.45500011e-02],
       [ 6.83513174e-02],
       [-3.94167952e-02],
       [-3.89371851e-02],
       [-8.61178318e-02],
       [-4.81173890e-02],
       [ 7.91015343e-02],
       [-4.66302847e-02],
       [-9.50259860e-02],
       [ 2.68525309e-03],
       [ 1.01875838e-01],
       [-1.03896759e-01],
       [-1.04336307e-01],
       [-9.7

In [83]:
e_1 = np.eye(30,1)


In [76]:
P.shape

(200, 30)

In [84]:
A.shape, e_1.shape

((30, 200), (30, 1))

In [95]:
e_1.T @ A @ A_0.T

array([[0.9584146]])

In [87]:
A_0 @ P @ e_1

array([[1.]])